In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/musical-instruments-sound-dataset/Metadata_Train.csv
/kaggle/input/musical-instruments-sound-dataset/Metadata_Test.csv
/kaggle/input/musical-instruments-sound-dataset/Test_submission/Test_submission/phaze-giant-bass-69687.wav
/kaggle/input/musical-instruments-sound-dataset/Test_submission/Test_submission/Sad-Violin-G-www.fesliyanstudios.com.wav
/kaggle/input/musical-instruments-sound-dataset/Test_submission/Test_submission/pop-drums-loops-3-11278.wav
/kaggle/input/musical-instruments-sound-dataset/Test_submission/Test_submission/paft-drunk-freestyle-drum-22227.wav
/kaggle/input/musical-instruments-sound-dataset/Test_submission/Test_submission/guitar-soundtrack-55233.wav
/kaggle/input/musical-instruments-sound-dataset/Test_submission/Test_submission/Va-ord-E4-ff-4c.wav
/kaggle/input/musical-instruments-sound-dataset/Test_submission/Test_submission/guitar-swellsolo-34241.wav
/kaggle/input/musical-instruments-sound-dataset/Test_submission/Test_submission/world-drum-beat-7621

# Imports and installs

In [15]:
#for ml
import os
import torch

#for audio
import librosa
import torchaudio


os.environ['KERAS_BACKEND']='torch'

import keras


In [2]:
!pip show keras

Name: keras
Version: 3.0.5
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, dm-tree, h5py, ml-dtypes, namex, numpy, rich
Required-by: keras-tuner, tensorflow


In [3]:
!pip freeze

absl-py==1.4.0
accelerate==0.27.2
access==1.1.9
affine==2.4.0
aiobotocore==2.11.2
aiofiles==22.1.0
aiohttp @ file:///home/conda/feedstock_root/build_artifacts/aiohttp_1701099469104/work
aiohttp-cors==0.7.0
aioitertools==0.11.0
aiorwlock==1.3.0
aiosignal @ file:///home/conda/feedstock_root/build_artifacts/aiosignal_1667935791922/work
aiosqlite==0.19.0
albumentations==1.4.0
alembic==1.13.1
altair==5.2.0
annotated-types==0.6.0
annoy==1.17.3
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1702909220329/work
apache-beam==2.46.0
aplus==0.11.0
appdirs==1.4.4
archspec @ file:///home/conda/feedstock_root/build_artifacts/archspec_1699370045702/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1695386546427/work
array-record==0.5.0
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1696128962909/work
arviz==0.17.0
astroid==3.0

# Create dataset and dataloader objects

- WORK NEEDED HERE

In [5]:
import pandas as pd

tr_data = pd.read_csv('/kaggle/input/musical-instruments-sound-dataset/Metadata_Train.csv')
te_data = pd.read_csv('/kaggle/input/musical-instruments-sound-dataset/Metadata_Test.csv')

In [7]:
#modify here
train_folder = '/kaggle/input/musical-instruments-sound-dataset/Train_submission/Train_submission/'
test_folder = '/kaggle/input/musical-instruments-sound-dataset/Test_submission/Test_submission/'

In [8]:
tr_data

,FileName,Class
0,1-E1-Major 00.wav,Sound_Guitar
1,1-E1-Major 01.wav,Sound_Guitar
2,1-E1-Major 02.wav,Sound_Guitar
3,1-E1-Major 03.wav,Sound_Guitar
4,1-E1-Major 04.wav,Sound_Guitar
...,...,...
2624,strange-piano-73881.wav,Sound_Piano
2625,the-last-piano-112677.wav,Sound_Piano
2626,this-is-war-version-e-95411.wav,Sound_Piano
2627,tired-ghosts-piano-65013.wav,Sound_Piano


In [11]:
tr_data.Class.unique()

array(['Sound_Guitar', 'Sound_Drum', 'Sound_Violin', 'Sound_Piano'],
      dtype=object)

In [19]:
#we need to improve it

print(tr_data.FileName[0])

librosa.load(train_folder + tr_data.FileName[0],sr=16000)

1-E1-Major 00.wav


(array([-1.4394413e-04, -2.8533602e-04, -3.0782184e-04, ...,
        -1.2475898e-06,  9.1803429e-07, -7.4332070e-07], dtype=float32),
 16000)

In [20]:
from IPython.display import Audio

au,sr = librosa.load(train_folder + tr_data.FileName[0],sr=16000)

Audio(au,rate=sr)

In [21]:
tr_data['FileName'] = train_folder + tr_data['FileName']

te_data['FileName'] = test_folder + te_data['FileName']

In [23]:
tr_data.loc[0]

FileName    /kaggle/input/musical-instruments-sound-datase...
Class                                            Sound_Guitar
Name: 0, dtype: object

In [40]:
#solving the val data problem
te_data.loc[te_data.Class == "Sound_Guiatr",'Class'] = "Sound_Guitar"

In [42]:
te_data

,FileName,Class
0,/kaggle/input/musical-instruments-sound-datase...,Sound_Guitar
1,/kaggle/input/musical-instruments-sound-datase...,Sound_Guitar
2,/kaggle/input/musical-instruments-sound-datase...,Sound_Guitar
3,/kaggle/input/musical-instruments-sound-datase...,Sound_Guitar
4,/kaggle/input/musical-instruments-sound-datase...,Sound_Guitar
...,...,...
75,/kaggle/input/musical-instruments-sound-datase...,Sound_Piano
76,/kaggle/input/musical-instruments-sound-datase...,Sound_Piano
77,/kaggle/input/musical-instruments-sound-datase...,Sound_Piano
78,/kaggle/input/musical-instruments-sound-datase...,Sound_Piano


In [26]:
labeldict = {index: element for index, element in enumerate(tr_data['Class'].unique())}

labeldict

{0: 'Sound_Guitar', 1: 'Sound_Drum', 2: 'Sound_Violin', 3: 'Sound_Piano'}

In [27]:
labeldict = {index: element for index, element in enumerate(tr_data['Class'].unique())}

def id2label(vec):
    for i in range(len(labeldict)):
        if labeldict[i]==vec:
            if os.environ['KERAS_BACKEND']=='torch':
                return torch.Tensor([i])            
            else:
                return tf.convert_to_tensor([i])

In [28]:
#using pytorch Dataset and Dataloader

from torch.utils.data import Dataset, DataLoader

class AudioDataset(Dataset):
    def __init__(self,path,label):
        self.path = path
        self.label = label
    
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self,index):
        audio_path = self.path[index]
        speech,sample_rate = librosa.load(audio_path,sr=16000)
        speech=torch.Tensor(speech)

        label = id2label(self.label[index])

        return audio_path,speech,sample_rate,label


In [43]:
train_dataset = AudioDataset(tr_data['FileName'],tr_data['Class'])
val_dataset = AudioDataset(te_data['FileName'],te_data['Class'])

In [44]:
val_dataset[0]

('/kaggle/input/musical-instruments-sound-dataset/Test_submission/Test_submission/acoustic-guitar-logo-13084.wav',
 tensor([-6.5331e-13, -1.9628e-13, -1.1489e-13,  ..., -1.9282e-12,
         -9.8003e-13,  2.7453e-13]),
 16000,
 tensor([0.]))

In [45]:
te_data['Class'].unique()

array(['Sound_Guitar', 'Sound_Drum', 'Sound_Violin', 'Sound_Piano'],
      dtype=object)

In [46]:
from torch.nn.functional import pad

transform = torchaudio.transforms.Spectrogram(n_fft=512)

def collate_fn(batch):
    #print(batch)
    path, waveforms, sample_rates, label = zip(*batch)
    # Calculate the maximum length within this batch
    max_length = max(len(waveform) for waveform in waveforms)
    #print(f'max_length = {max_length}')
    # Pad all sequences to match the maximum length
    padded_waveforms = torch.stack([torch.transpose(transform(pad(waveform,(0, max_length - len(waveform)), mode='constant', value=0)),0,1) for waveform in waveforms])
    
    #fft
    padded_waveforms = torch.unsqueeze(padded_waveforms,-1)
    
    #print(f'length of the largest audio: {len(padded_waveforms[0])}')
    
    return padded_waveforms,torch.stack(label)

In [47]:
BATCH_SIZE = 4 #multiples of 2

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)


for batch_idx, (audio_batch, label_batch) in enumerate(train_dataloader):
    print(f"Batch #{batch_idx}")
    print(f"\tAudio batch:\n{audio_batch.shape}\n")
    print(f"\tLabel batch:\n{label_batch}\n")
    break

Batch #0
	Audio batch:
torch.Size([4, 2216, 257, 1])

	Label batch:
tensor([[2.],
        [0.],
        [3.],
        [1.]])



# Create Model

In [48]:
inputs = keras.Input(shape=(None,257,1))

x = keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = keras.layers.MaxPooling2D(pool_size=2)(x) 
x = keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x) 
x = keras.layers.MaxPooling2D(pool_size=2)(x) 
x = keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = keras.layers.MaxPooling2D(pool_size=2)(x) 
x = keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x) 
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(len(labeldict),'softmax')(x)

model_fft2 = keras.Model(inputs=inputs,outputs=outputs)

model_fft2.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, 257, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, None, 255, 32)  │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, None, 127, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, None, 125, 64)  │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, None, 62, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, None, 60, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, None, 30, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, None, 28, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 388,868 (1.48 MB)

 Trainable params: 388,868 (1.48 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
model_fft2.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

history = model_fft2.fit(train_dataloader,
                     validation_data=val_dataloader,
                     epochs=10,
                     batch_size=BATCH_SIZE)

Epoch 1/10
 37/658 ━━━━━━━━━━━━━━━━━━━━ 46:11 4s/step - accuracy: 0.2819 - loss: 1.6362

KeyboardInterrupt: 

In [ ]:
#plot the training results!!!

import matplotlib.pyplot as plt 


accuracy = history.history["accuracy"] 
val_accuracy = history.history["val_accuracy"] 
loss = history.history["loss"] 
val_loss = history.history["val_loss"] 

epochs = range(1, len(accuracy) + 1) 

plt.figure(figsize=(14, 7))
plt.subplot(121)
plt.plot(epochs, accuracy, "bo", label="Training accuracy") 
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.legend()

plt.subplot(122)
plt.plot(epochs, loss, "bo", label="Training loss") 
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.legend()

# Evaluation

In [51]:
def model_prediction(audio):
    
    #transform via FFT
    transform = torchaudio.transforms.Spectrogram(n_fft=512)
    audio = transform(audio)
    #modify dimensions so it conforms to model
    audio = audio.transpose(0,1)
    conformed_audio = audio.expand(1,audio.size()[0],audio.size()[1])
    
    #predict class
    predictions = model_fft2.predict(conformed_audio)[0]
    
    #get class with biggest score
    chosen = torch.argmax(torch.Tensor(predictions))
    
    string_chosen = labeldict[int(chosen.numpy())]
    return string_chosen

In [53]:
tr_data.FileName[0]

'/kaggle/input/musical-instruments-sound-dataset/Train_submission/Train_submission/1-E1-Major 00.wav'

In [52]:
au,sr = librosa.load(tr_data.FileName[0])

au = torch.Tensor(au)

model_prediction(au)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


'Sound_Drum'

In [58]:
#simple prediction
audio = train_dataset[0][1]
audio

tensor([-1.4394e-04, -2.8534e-04, -3.0782e-04,  ..., -1.2476e-06,
         9.1803e-07, -7.4332e-07])

In [62]:
transform = torchaudio.transforms.Spectrogram(n_fft=512)
#audio = transform(audio)

audio.size()

torch.Size([257, 157])

In [63]:
audio = audio.transpose(0,1)
conformed_audio = audio.expand(1,audio.size()[0],audio.size()[1])

conformed_audio.size()

torch.Size([1, 157, 257])

In [66]:
predictions = model_fft2.predict(conformed_audio)[0]
print(predictions)
chosen = torch.argmax(torch.Tensor(predictions))
print(chosen)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[0.23837338 0.26833683 0.2690641  0.22422574]
tensor(2)


In [67]:
labeldict[int(chosen.numpy())]

'Sound_Violin'

# Presenting in an interface

- WORK NEEDED HERE

In [69]:
!pip install gradio
#interesting link for gradio: https://www.gradio.app/guides/quickstart

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 59.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 79.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.4 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=26de36ea3f983eedb2fcb134f3a030d00b61c92be02d42b41b917fdfbec66bb9
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.12.3
    Uninstalling tomlkit-0.12.3:
      Successfully uninstalled tomlkit-0.12.3


In [70]:
import gradio as gr

In [1]:
demo = gr.Interface(
    fn=model_prediction,
    inputs=["audio"],
    outputs=["text"],
)

demo.launch(share=True,debug=True)

NameError: name 'gr' is not defined